Importing libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

print('Libraries imported.')

Libraries imported.


In [3]:
pip install beautifulsoup4 as bs4

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup

In [6]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests

Reading the data from the url provided

In [8]:
### Reading the data from Toronto Wiki
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

In [9]:
### Convert the html data
data = response.text
soup = BeautifulSoup(data,'html.parser')
wiki_table=soup.find('table')

In [10]:
### Dataframe using pandas
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
df.head()

,Postalcode,Borough,Neighborhood


In [11]:
for tr_cell in wiki_table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [12]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [35]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
#Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

Removing rows with borough "Not assigned"

In [13]:
# Removing rows with borough "Not assigned"

df2=df[df['Borough'].str.contains("Not assigned") == False].reset_index()
df2.head()

,index,Postalcode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Grouping the data by Postalcode and borough

In [14]:
# Group them by postal code and borough
df3= df2.groupby(['Postalcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Using .shape to find the no. of rows of the dataframe.

In [15]:
df3.shape

(103, 3)

In [16]:
print('The number of rows in the of the dataframe is 103.')

The number of rows in the of the dataframe is 103.


In [17]:
#Save data as'Toronto1.csv'
df3.to_csv('Toronto1.csv')
print('Successfully Saved!')

Successfully Saved!


Segmenting and Clustering Neighborhoods in Toronto part 2

Using the csv file provide from http://cocl.us/Geospatial_data instead of Geocoder.



In [18]:
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_latlon=pd.read_csv(url)
df_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


To make the column similar to the "PostalCode" in the example without a space.

In [38]:
df_latlon = df_latlon.rename(columns = {'PostalCode':'Postalcode'}) 
df_latlon.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df3=pd.read_csv('Toronto1.csv')
df3.head()

,Unnamed: 0,Postalcode,Borough,Neighborhood
0,0,M1B,Scarborough,"Malvern, Rouge"
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


In [33]:
del df3['Unnamed: 0']
df3.head(12)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [39]:
df3 = pd.merge(df3, df_latlon, on = 'Postalcode')
df3.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [40]:
df3.shape

(103, 5)

Since the data in the dataframe are grouped by postal code and borough, they may look different from the picture but the data within should be the same, just in different order

In [41]:
df3.to_csv('Toronto2.csv', index=False)
print('Successfully Saved!')

Successfully Saved!
